In [63]:
import pandas as pd
import numpy as np
from pywebio.input import *
from pywebio.output import *

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix

In [4]:
def readFile(diag_fn, labs_fn):
    diag_file = open(diag_fn)
    labs_fn = open(labs_fn)
    
    diagnosis_admissions_table = pd.read_table(diag_fn)
    
    labs_table = pd.read_table(labs_fn)
    labs_table['PatientID'] =  labs_table['PatientID'] + '_' + labs_table['AdmissionID'].astype(str)
            
    return diagnosis_admissions_table, labs_table

In [5]:
diag_table, labs_table = readFile('AdmissionsDiagnosesCorePopulatedTable.txt', 'LabsCorePopulatedTable.txt')

In [6]:
labs_table

,PatientID,AdmissionID,LabName,LabValue,LabUnits,LabDateTime
0,BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1,1,METABOLIC: ALT/SGPT,40.0,U/L,1974-01-11 21:10:51.930000000
1,BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1,1,CBC: MONOCYTES,0.9,k/cumm,1974-01-11 03:14:30.127000000
2,BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1,1,METABOLIC: GLUCOSE,134.2,mg/dL,1974-01-11 13:57:06.970000000
3,BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1,1,CBC: RDW,15.0,%,1974-01-11 02:10:43.437000000
4,BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1,1,METABOLIC: ALBUMIN,2.8,gm/dL,1974-01-11 18:50:32.400000000
...,...,...,...,...,...,...
107535382,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C_3,3,URINALYSIS: SPECIFIC GRAVITY,1.0,no unit,2009-04-13 00:52:38.720000000
107535383,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C_3,3,CBC: MCH,35.3,pg,2009-04-12 22:55:23.240000000
107535384,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C_3,3,METABOLIC: AST/SGOT,28.9,U/L,2009-04-12 23:43:42.950000000
107535385,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C_3,3,CBC: RED BLOOD CELL COUNT,3.2,m/cumm,2009-04-12 23:04:07.557000000


In [7]:
diag_table.sort_values('PatientID', inplace=True)
total_diag = diag_table['PrimaryDiagnosisDescription'] # for with details (ex: Protozoal diseases complicating pregnancy, first trimester)

In [8]:
top_diseases =['Pathological fracture in neoplastic disease', 'Bipolar disorder', 'Hydroxyapatite deposition disease', 'Rheumatoid nodule', 'Kaschin-Beck disease']

# Setting x-matrix

In [12]:
grouped_labs = labs_table.groupby(['PatientID', 'LabName']).aggregate(np.mean)
og_labs = grouped_labs.reset_index()

In [13]:
x = og_labs.pivot(index='PatientID', columns='LabName', values='LabValue').sort_index()

# Saving dataframes

In [14]:
x.to_pickle("x_data.pkl")
total_diag.to_pickle("y_data.pkl")

# x.read_pickle("x_data.pkl")
# total_diag.read_pickle("x_data.pkl")

# Setting y-matrix

In [ ]:
disease = input() # ex: neoplasm

In [ ]:
y = []

for diagnosis in total_diag:
  if disease in diagnosis:
    y.append(True)
  else: 
    y.append(False)

# Comparison logic

In [32]:
# comparing similarity between every patient
# reminder: figure out what to do with NaNs (maybe remove those tests when doing comparisons)

# this is comparing the LabValue?

def compareSimilar(patientId1, patientId2):
    
    patientLabs1 = x.loc[patientId1]
    patientLabs2 = x.loc[patientId2]

    return 1 - np.mean(abs((patientLabs1 - patientLabs2)))

In [37]:
compareSimilar('BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1', 'BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1')

1.0

In [62]:
x.loc['BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1'] #['LabValue']

LabName
CBC: ABSOLUTE LYMPHOCYTES         29.466667
CBC: ABSOLUTE NEUTROPHILS         72.070000
CBC: BASOPHILS                     0.133333
CBC: EOSINOPHILS                   0.360000
CBC: HEMATOCRIT                   42.240000
CBC: HEMOGLOBIN                   14.428571
CBC: LYMPHOCYTES                   2.150000
CBC: MCH                          28.200000
CBC: MCHC                         33.510000
CBC: MEAN CORPUSCULAR VOLUME      85.800000
CBC: MONOCYTES                     0.711111
CBC: NEUTROPHILS                   5.633333
CBC: PLATELET COUNT              259.100000
CBC: RDW                          12.430000
CBC: RED BLOOD CELL COUNT          4.930769
CBC: WHITE BLOOD CELL COUNT        6.284615
METABOLIC: ALBUMIN                 3.781818
METABOLIC: ALK PHOS              117.920000
METABOLIC: ALT/SGPT               37.209091
METABOLIC: ANION GAP               7.910000
METABOLIC: AST/SGOT               29.090909
METABOLIC: BILI TOTAL              0.512500
METABOLIC: BUN          

In [83]:

cbc_tests = {'Absolute Lymphocytes':1, 'Absolute Neutrophils':2, 'Basophils':5, 'Eosinophils':3, 'Hematocrit':5, 'Hemoglobin':2, 'Lymphocytes':2, 'MCH':5, 'MCHC':1, 'Mean Corpuscular Volume':5, 'Monocytes':9, 'Neutrophils':4, 'Platelet Count':2, 'RDW':5, 'Red Blood Cell Count':2, 'White Blood Cell Count':5} 
 
metabolic_tests = {'Albumin':4, 'ALK PHOS':1, 'ALT/SGPT':5, 'ANION GAP':1, 'AST/SGOT':6, 'BILI Total':2, 'BUN':6, 'Calcium':2, 'Carbon Dioxide':6, 'Chloride':1, 'Creatinine':4, 'Glucose':2, 'Potassium':7, 'Sodium':2, 'Total Protein':6} 

urinalysis_tests = {'pH':7, 'Red Blood Cells':4, 'Specific Gravity':4, 'White Blood Cells':7}
    

In [84]:
urinalysis_tests

{'pH': 7, 'Red Blood Cells': 4, 'Specific Gravity': 4, 'White Blood Cells': 7}

In [85]:
all_tests = []

for key, val in cbc_tests.items():
    all_tests.append(val)
    
for key, val in metabolic_tests.items():
    all_tests.append(val)
    
for key, val in urinalysis_tests.items():
    all_tests.append(val)

In [86]:
all_tests
    

[1,
 2,
 5,
 3,
 5,
 2,
 2,
 5,
 1,
 5,
 9,
 4,
 2,
 5,
 2,
 5,
 4,
 1,
 5,
 1,
 6,
 2,
 6,
 2,
 6,
 1,
 4,
 2,
 7,
 2,
 6,
 7,
 4,
 4,
 7]

In [87]:
def compareSimilar2(patientId1, new_patient_labs):
    
    patientLabs1 = x.loc[patientId1]

    return 1 - np.mean(abs((patientLabs1 - new_patient_labs)))

In [88]:
compareSimilar2('BF7165BC-CCD9-4CE4-944F-E3C9786B7BF3_1', all_tests)

-31.839540776683634

In [ ]:
# edge cases: patient w a lot missing data, 
#             patient that is exactly the same as another patient, patient that has all missing data
# enter text instead of numerical value, can any values be -ve?, are units consistent?




In [68]:
cbc_tests

{'Absolute Lymphocytes': 1,
 'Absolute Neutrophils': 2,
 'Basophils': 5,
 'Eosinophils': 3,
 'Hematocrit': 5,
 'Hemoglobin': 2,
 'Lymphocytes': 2,
 'MCH': 5,
 'MCHC': 1,
 'Mean Corpuscular Volume': 5,
 'Monocytes': 9,
 'Neutrophils': 4,
 'Platelet Count': 2,
 'RDW': 5,
 'Red Blood Cell Count': 2,
 'White Blood Cell Count': 5}